In [1]:
using Plots

In [121]:
eps = matread("test_iplp.mat")["eps"]
A = matread("test_iplp.mat")["A"]
c = matread("test_iplp.mat")["c"]
b = matread("test_iplp.mat")["b"]
eta = matread("test_iplp.mat")["eta"]
kmax = matread("test_iplp.mat")["kmax"]

m, n = size(A)
@assert size(c, 1) == n
@assert size(b, 1) == m
m, n

(5, 7)

In [122]:
module Optimizers
using LinearAlgebra
function ipm_lm(A, b, c, eta, eps, kmax, x0, lambda0, s0)
    m, n = size(A)
    @assert size(c, 1) == n
    @assert size(b, 1) == m
    e = ones(n)
    
    k = 0
    
    xk = x0
    lambdak = lambda0
    sk = s0
    etak = eta
    
    mu0 = (xk' * sk) / n
    
    while true
        if k > kmax
            break
        end
        
        muk = (xk' * sk) / n
        
        if muk <= eps*mu0
            break
        end
        
        F(x, lambda, s) = [
            A' * lambda + s - c
            A * x - b
            diagm(x) * diagm(s) * e
        ]
        JF(x, lambda, s) = [
            zeros(n, n)              A'           I(n)
                      A     zeros(m, m)    zeros(m, n)
                diagm(s)    zeros(n, m)       diagm(x)
        ]

        # 1. (Prediction)
        affine_scaling_step_k = JF(xk, lambdak, sk) \ -F(xk, lambdak, sk)
        delta_xk_aff = affine_scaling_step_k[1:n]
        delta_lambdak_aff = affine_scaling_step_k[n+1:n+m]
        delta_sk_aff = affine_scaling_step_k[n+m+1:n+m+n]

        # 2.
        alpha_P_aff = min(1, min([(-(xk[i])/delta_xk_aff[i]) for i in 1:n if delta_xk_aff[i] < 0]...))
        alpha_D_aff = min(1, min([(-(sk[i])/delta_sk_aff[i]) for i in 1:n if delta_sk_aff[i] < 0]...))

        # 3.
        muk_aff = ((xk + alpha_P_aff * delta_xk_aff)' * (sk + alpha_D_aff * delta_sk_aff)) / n
        sigmak = (muk_aff / muk)^3

        # 4. (Correction)
        corrector_step_k = JF(xk, lambdak, sk) \ (-F(xk, lambdak, sk) + [
                zeros(n)
                zeros(m)
                -diagm(delta_xk_aff)*diagm(delta_sk_aff)*e + sigmak * muk * e
        ])
        delta_xk = corrector_step_k[1:n]
        delta_lambdak = corrector_step_k[n+1:n+m]
        delta_sk = corrector_step_k[n+m+1:n+m+n]

        # 5.
        alpha_P_hat = min(1, min([(-(xk[i])/delta_xk[i]) for i in 1:n if delta_xk[i] < 0]...))
        alpha_D_hat = min(1, min([(-(sk[i])/delta_sk[i]) for i in 1:n if delta_sk[i] < 0]...))

        alpha_P = min(1, etak * alpha_P_hat)
        alpha_D = min(1, etak * alpha_D_hat)

        # 6.
        xk += alpha_P * delta_xk
        lambdak += alpha_D * delta_lambdak
        sk += alpha_D * delta_sk
        
        k += 1
    end
    
    muk = (xk' * sk) / n
    fk = c' * xk
    
    return xk, lambdak, sk, fk, muk, k
end
end
using Main.Optimizers

x0 = ones(n)
lambda0 = ones(m)
s0 = ones(n)

xk, lambdak, sk, fk, muk, k = Optimizers.ipm_lm(A, b, c, eta, eps, kmax, x0, lambda0, s0)

@show xk
@show lambdak
@show sk
@show fk
@show muk
@show k

nothing

xk = [4.012053859498594e-7, 3.151871597335268e-7, 9.999998963709144, 15.99999887297977, 3.000000084447843, 4.012053859498602e-7, 3.1518715973353103e-7]
lambdak = [7.607195153767005e-8, -1.1590980448187494e-8, 8.132554521518028e-10, -0.49999995419098814, -0.49999994908755585]
sk = [0.5000000332735678, 0.5000000281701388, 3.0106303361580557e-9, 9.06735623220153e-8, 7.826932642167679e-8, 0.50000003327357, 0.5000000281701378]
fk = [7.163925456833862e-7]
muk = 3.4744053874541354e-7
k = 7
